In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.gridspec import GridSpec

sns.set_context('talk')
sns.set_style("whitegrid")
sns.set_palette("colorblind")

import copy
from hendrics.fake import filter_for_deadtime
from stingray import Powerspectrum, Crossspectrum, AveragedPowerspectrum, AveragedCrossspectrum

from astropy.modeling.models import Lorentz1D, Const1D
import numpy as np
import scipy.stats
import scipy.special
import scipy.fftpack
from scipy.interpolate import interp1d
from stingray.utils import rebin_data_log
from stingray.gti import bin_intervals_from_gtis
from hendrics.base import r_in, r_det
from stingray.modeling.posterior import GaussianLogLikelihood
from stingray.modeling.parameterestimation import ParameterEstimation
import os
import glob

from stingray.utils import rebin_data

np.random.seed(348172694)

from stingray import Lightcurve, Crossspectrum
from stingray.simulator.simulator import Simulator
from scipy.ndimage.filters import gaussian_filter1d

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## What about different fluxes?

In [3]:
from hendrics.io import load_events, load_lcurve, load_pds, save_lcurve, save_pds
from stingray.gti import cross_two_gtis
from stingray import Powerspectrum, Crossspectrum, AveragedPowerspectrum, AveragedCrossspectrum

In [4]:
from astropy.modeling.models import Lorentz1D
from stingray.events import EventList

def create_flat_lightcurve(tstart=0, tstop=8192, dt=1/512, 
                            mean_ctrate=200, rms=0.25): 

    time = np.arange(tstart, tstop, dt)

    # The current normalization is not precise for very high count rates!
    sim_lc_clean = Lightcurve(time, mean_ctrate + np.zeros_like(time))
    
    sim_lc_clean.counts *= dt

    return sim_lc_clean


In [5]:
def create_event_lists_from_lightcurve(lc_in, channel_ratio=1):
    # work on a copy, not on the original light curve
    lc = copy.deepcopy(lc_in)
    events = EventList()
    events.simulate_times(lc)
    event_times1 = events.time

    lc.counts *= channel_ratio
    events.simulate_times(lc)
    event_times2 = events.time
    return event_times1, event_times2

def create_lightcurve_with_deadtime(event_times, dt, tstart=0, tstop=8192, 
                                    deadtime=2.5e-3):
    gti = [[tstart, tstop]]
    lc_clean = Lightcurve.make_lightcurve(event_times, 
                                          gti=gti, 
                                          tstart=tstart, tseg=tstop - tstart,
                                          dt=dt)
    events_dt = filter_for_deadtime(event_times, deadtime)
    lc = Lightcurve.make_lightcurve(events_dt, 
                                    gti=gti, 
                                    tstart=tstart, tseg=tstop - tstart,
                                    dt=dt)
    return lc_clean, lc

   
def create_lightcurves_with_deadtime(lc_in, channel_ratio=1, 
                                     lc_dt=1/2048, deadtime=2.5e-3):
    
    event_times1, event_times2 = \
        create_event_lists_from_lightcurve(lc_in, channel_ratio=channel_ratio)

    lc1_clean, lc1 = \
        create_lightcurve_with_deadtime(event_times1, lc_dt, 
                                        tstart=lc_in.tstart, 
                                        tstop=lc_in.tseg - lc_in.tstart,
                                        deadtime=deadtime)
    lc2_clean, lc2 = \
        create_lightcurve_with_deadtime(event_times2, lc_dt, 
                                        tstart=lc_in.tstart, 
                                        tstop=lc_in.tseg - lc_in.tstart,
                                        deadtime=deadtime)

    return lc1_clean, lc1, lc2_clean, lc2

def calculate_and_save_freq_spectrum(lc1, lc2=None, segment_size=512, 
                                     file_name=None, norm='frac'):
    if lc2 is None:
        pds = AveragedPowerspectrum(lc1, segment_size=segment_size, 
                                    norm=norm)
        if file_name is None:
            file_name = 'pds.nc'
    else:
        pds = AveragedCrossspectrum(lc1, lc2, 
                                    segment_size=segment_size, 
                                    norm=norm)
        if file_name is None:
            file_name = 'cpds.nc'

#     print('Saving ' + file_name)
    save_pds(pds, file_name, save_all=False)

    return file_name

def modify_saved_power(in_file_name, new_power, meancounts=None, 
                       out_file_name=None):
    
    if out_file_name is None:
        out_file_name = in_file_name.replace('.nc', '_out.nc')
    newpow = load_pds(in_file_name)

    newpow.norm = 'none'
    newpow.power = new_power
    if meancounts is not None:
        newpow.power = newpow.power / newpow.n**2 
        newpow.power = newpow.power * 2 * newpow.segment_size / meancounts **2

#     print('Saving ' + out_file_name)
    save_pds(newpow, out_file_name)


In [6]:
def calculate_fad_and_save_all_spectra(lc1, lc2, file_label='all_',
                                       segment_size=512, 
                                       smoothing_alg='gauss', 
                                       smoothing_length=None):
    from fad_correction import FAD_power_spectra

    freqs, pds1_norm, pds2_norm, crosssp_norm, ptot_norm = \
        FAD_power_spectra(lc1, lc2, segment_size=segment_size, smoothing_alg=smoothing_alg, 
                          smoothing_length=smoothing_length)

#     print('Fourier Amplitudes loaded')
    calculate_and_save_freq_spectrum(lc1, segment_size=segment_size, 
                                     file_name=file_label + 'pds1_dt.nc')
    calculate_and_save_freq_spectrum(lc2, segment_size=segment_size, 
                                     file_name=file_label + 'pds2_dt.nc')
    modify_saved_power(file_label + 'pds1_dt.nc', ptot_norm, lc1.meancounts + lc2.meancounts, 
                       out_file_name=file_label + 'pdstot_norm.nc')
    modify_saved_power(file_label + 'pds1_dt.nc', pds1_norm, lc1.meancounts, 
                       out_file_name=file_label + 'pds1_norm.nc')
    calculate_and_save_freq_spectrum(lc1, lc2, segment_size=segment_size, 
                                     file_name=file_label + 'cs_dt.nc')
    modify_saved_power(file_label + 'pds1_dt.nc', crosssp_norm, (lc1.meancounts + lc2.meancounts) / 2, 
                       out_file_name=file_label + 'cs_norm.nc')

In [7]:
def load_rebinned(file, rebin_factor, kind='lin'):
    periodogram = load_pds(file)
    if kind == 'lin' and isinstance(rebin_factor, int):
        return periodogram.rebin(f=rebin_factor)
    elif kind == 'lin':
        return periodogram.rebin(df=rebin_factor)
    else:
        return periodogram.rebin_log(rebin_factor)
    

In [15]:
def simulate_flat_spectrum(channel_ratio=1, rebin_factor=0.02, segment_size=512,
                      label=None, tstart = 0, tstop = 8192, smoothing_alg='gauss',
                      smoothing_length=None, rms=0.25, mean_ctrate=200):
    

    sim_lc = create_flat_lightcurve(tstart, tstop, rms=rms,
                                     mean_ctrate=mean_ctrate)
    lc1_clean, lc1, lc2_clean, lc2 = \
        create_lightcurves_with_deadtime(sim_lc, 
                                         channel_ratio=channel_ratio, 
                                         lc_dt=1/2048, deadtime=2.5e-3)
    if label is None:
        label = '{:.2f}_flat'.format(channel_ratio * 100)
    calculate_and_save_freq_spectrum(lc1_clean, segment_size=segment_size, 
                                     file_name=label + 'pds1_clean.nc')

    calculate_and_save_freq_spectrum(lc1_clean, lc2_clean, segment_size=segment_size, 
                                     file_name=label + 'cs_clean.nc')
    
    calculate_fad_and_save_all_spectra(lc1, lc2, file_label=label, 
                                       segment_size=segment_size,
                                       smoothing_alg=smoothing_alg, 
                                       smoothing_length=smoothing_length)
    return label


In [16]:
label = simulate_flat_spectrum(tstop=1024, segment_size=512, smoothing_alg='gauss')

In [17]:
def plot_spectrum(file_label, rebin_factor=0.02, step=0.,
                  white_fmin=0, white_fmax=2048, xlim=None, ylim=None):
    cs_dt_reb = load_rebinned(file_label + 'cs_dt.nc', rebin_factor)
    cs_norm_reb = load_rebinned(file_label + 'cs_norm.nc', rebin_factor)
    pds_norm_reb = load_rebinned(file_label + 'pds1_norm.nc', rebin_factor)
    pdstot_norm_reb = load_rebinned(file_label + 'pdstot_norm.nc', rebin_factor)
    pds1_dt_reb = load_rebinned(file_label + 'pds1_dt.nc', rebin_factor)
    
    lw=2
    good = (pds1_dt_reb.freq >= white_fmin)&(pds1_dt_reb.freq < white_fmax)
    fig = plt.figure()
    plt.plot(pds1_dt_reb.freq, pds1_dt_reb.power - np.mean(pds1_dt_reb.power[good]), 
             label='PDS 1', drawstyle='steps-mid', zorder=10, lw=lw)
    plt.plot(pds_norm_reb.freq, pds_norm_reb.power - np.mean(pds_norm_reb.power[good]) + step, 
             label='PDS 1, FAD-normalized', drawstyle='steps-mid', lw=lw, color='r')
    plt.plot(pdstot_norm_reb.freq, pdstot_norm_reb.power - np.mean(pdstot_norm_reb.power[good]) + 2*step, 
             label='PDS 1+2, FAD-normalized', drawstyle='steps-mid', lw=lw, color='b')
    plt.plot(cs_dt_reb.freq, cs_dt_reb.power + 3 * step, 
             label='Cospectrum', drawstyle='steps-mid', lw=lw, color='grey')
    plt.plot(cs_norm_reb.freq, cs_norm_reb.power + 4 * step, 
             label='Cospectrum, FAD-normalized', drawstyle='steps-mid', lw=lw, color='k')
    plt.legend(loc=1)

    if step > 0:
        for i in range(4):
            plt.axhline(step * i, ls = '--', color='k', zorder=20)

    plt.xlim(xlim)
    plt.xlabel('Frequency (Hz)')
    plt.ylabel(r'${(\rm rms/mean)}^2\nu^{-1}$')

    plt.savefig(file_label + 'rebin.jpg')
    plt.close(fig)

def plot_spectrum_log(file_label, rebin_factor=0.02, step=0.,
                      white_fmin=0, white_fmax=2048, xlim=None, ylim=None):
    cs_dt_reb_log = load_rebinned(file_label + 'cs_dt.nc', rebin_factor, kind='log')
    cs_norm_reb_log = load_rebinned(file_label + 'cs_norm.nc', rebin_factor, kind='log')
    pds_norm_reb_log = load_rebinned(file_label + 'pds1_norm.nc', rebin_factor, kind='log')
    pdstot_norm_reb_log = load_rebinned(file_label + 'pdstot_norm.nc', rebin_factor, kind='log')
    pds1_dt_reb_log = load_rebinned(file_label + 'pds1_dt.nc', rebin_factor, kind='log')
    lw=2
    fig, ax1 = plt.subplots()

    ax1.plot(cs_dt_reb_log.freq, cs_dt_reb_log.freq*cs_dt_reb_log.power, 
             label='Cospectrum', 
             lw=lw, drawstyle='steps-mid', color='grey', zorder=3)

    ax1.plot(cs_norm_reb_log.freq, cs_norm_reb_log.freq*cs_norm_reb_log.power, 
             label='Cospectrum, FAD-normalized', 
             lw=lw, drawstyle='steps-mid', color='k')

    good = (pds1_dt_reb_log.freq >= white_fmin)&(pds1_dt_reb_log.freq < white_fmax)

    lev = np.mean(pds_norm_reb_log.power.real[good])
    ax1.plot(pds_norm_reb_log.freq,
             pds_norm_reb_log.freq*(pds_norm_reb_log.power - lev), 
             alpha=0.6, label='PDS 1, FAD-normalized', 
             lw=lw, drawstyle='steps-mid', color='r')

    lev = np.mean(pdstot_norm_reb_log.power.real[good])
    ax1.plot(pdstot_norm_reb_log.freq, 
             pdstot_norm_reb_log.freq*(pdstot_norm_reb_log.power - lev), 
             alpha=0.6, label='PDS 1, FAD-normalized', 
             lw=lw, drawstyle='steps-mid', color='b')

    ax1.legend()
    ax1.set_yscale('log', nonposy='clip')

    ax1.set_xscale('log', nonposx='clip')

    ax1.set_xlim(xlim)
    ax1.set_ylim(ylim)

    ax1.set_xlabel('Frequency (Hz)')
    ax1.set_ylabel(r'${(\rm rms/mean)}^2$')

    plt.savefig(file_label + 'rebin_log.jpg')
    plt.close(fig)

In [20]:
plot_spectrum('100.00_flat', rebin_factor=1, step=0., xlim=None, ylim=None)

## And now, some statistics.

### TODO: Reliably normalize spectra in Leahy normalization and evaluate distribution with chi squared and Laplace.